# CIFAR-10 Challange - 김영인

1. resnet152
2. \+ Data Augmentation(RandomResizedCrop, RandomHorizontalFlip)
3. \+ Data Normalize
4. \+ regularization(dropout, weight decay)
5. \+ Optimizer(momentum, scheduler)


In [1]:
import random

import torch
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = device0
print(f"device: {device}") if torch.cuda.is_available() else print("device: cpu")

device: cuda:0


## Hyper parameter Setting 

In [2]:
learning_rate = 0.001
training_epochs = 50
batch_size = 32
momentum = 0.9
scheduler_step = 1
scheduler_gamma = 0.96

## Load & Preprocess Data

In [3]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224), # data augmentation, 224: image size, ImageNet pretrained model에 맞추기 위해서 224 size로 설정
     transforms.RandomHorizontalFlip(), # data augmentation, 좌우로 대칭
     transforms.ToTensor(), # numpy array를 pytorch tensor로 바꿔주는 역할
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # dataset의 mean, std를 이용해서 -1~1 로 normalize
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load and Define pretrained model and fine-tuning

In [4]:
class ResNet152(nn.Module):
    def __init__ (self):
        super(ResNet152, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.resnet(x) # batch_size * 10
        return x

## Load Model

In [5]:
net = ResNet152()
net = net.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size = scheduler_step,
                                      gamma = scheduler_gamma)

## Train

In [6]:
net.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
        if i % 750 == 0:
            class_correct = np.zeros(10)
            class_total = np.zeros(10)

            net.eval()

            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs, 1)
                    c = (predicted == labels).squeeze()
                    for i in range(4):
                        label = labels[i]
                        class_correct[label] += c[i].item()
                        class_total[label] += 1
            print('==================')
            print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))
            print('==================')
            
            net.train()
        
print('Finished Training')

Mean Accuracy :  8.63
[1,   100] loss: 1.761
[1,   200] loss: 0.976
[1,   300] loss: 0.817
[1,   400] loss: 0.734
[1,   500] loss: 0.665
[1,   600] loss: 0.653
[1,   700] loss: 0.611
Mean Accuracy :  81.43
[1,   800] loss: 0.591
[1,   900] loss: 0.581
[1,  1000] loss: 0.573
[1,  1100] loss: 0.534
[1,  1200] loss: 0.537
[1,  1300] loss: 0.551
[1,  1400] loss: 0.518
[1,  1500] loss: 0.528
Mean Accuracy :  83.35
Mean Accuracy :  83.98
[2,   100] loss: 0.469
[2,   200] loss: 0.472
[2,   300] loss: 0.465
[2,   400] loss: 0.469
[2,   500] loss: 0.457
[2,   600] loss: 0.445
[2,   700] loss: 0.480
Mean Accuracy :  84.8
[2,   800] loss: 0.466
[2,   900] loss: 0.448
[2,  1000] loss: 0.431
[2,  1100] loss: 0.441
[2,  1200] loss: 0.454
[2,  1300] loss: 0.453
[2,  1400] loss: 0.460
[2,  1500] loss: 0.453
Mean Accuracy :  86.45
Mean Accuracy :  86.24
[3,   100] loss: 0.403
[3,   200] loss: 0.410
[3,   300] loss: 0.397
[3,   400] loss: 0.402
[3,   500] loss: 0.422
[3,   600] loss: 0.424
[3,   700] lo

[16,   700] loss: 0.226
Mean Accuracy :  89.05
[16,   800] loss: 0.245
[16,   900] loss: 0.246
[16,  1000] loss: 0.234
[16,  1100] loss: 0.238
[16,  1200] loss: 0.244
[16,  1300] loss: 0.282
[16,  1400] loss: 0.241
[16,  1500] loss: 0.244
Mean Accuracy :  89.55
Mean Accuracy :  90.1
[17,   100] loss: 0.235
[17,   200] loss: 0.222
[17,   300] loss: 0.252
[17,   400] loss: 0.238
[17,   500] loss: 0.217
[17,   600] loss: 0.242
[17,   700] loss: 0.241
Mean Accuracy :  89.63
[17,   800] loss: 0.231
[17,   900] loss: 0.253
[17,  1000] loss: 0.233
[17,  1100] loss: 0.257
[17,  1200] loss: 0.223
[17,  1300] loss: 0.240
[17,  1400] loss: 0.238
[17,  1500] loss: 0.249
Mean Accuracy :  89.21
Mean Accuracy :  89.94
[18,   100] loss: 0.230
[18,   200] loss: 0.238
[18,   300] loss: 0.235
[18,   400] loss: 0.244
[18,   500] loss: 0.244
[18,   600] loss: 0.225
[18,   700] loss: 0.227
Mean Accuracy :  89.76
[18,   800] loss: 0.234
[18,   900] loss: 0.225
[18,  1000] loss: 0.239
[18,  1100] loss: 0.233


[31,   800] loss: 0.213
[31,   900] loss: 0.217
[31,  1000] loss: 0.206
[31,  1100] loss: 0.209
[31,  1200] loss: 0.195
[31,  1300] loss: 0.193
[31,  1400] loss: 0.190
[31,  1500] loss: 0.199
Mean Accuracy :  90.32
Mean Accuracy :  90.39
[32,   100] loss: 0.201
[32,   200] loss: 0.195
[32,   300] loss: 0.194
[32,   400] loss: 0.212
[32,   500] loss: 0.191
[32,   600] loss: 0.214
[32,   700] loss: 0.199
Mean Accuracy :  90.4
[32,   800] loss: 0.196
[32,   900] loss: 0.180
[32,  1000] loss: 0.192
[32,  1100] loss: 0.199
[32,  1200] loss: 0.190
[32,  1300] loss: 0.194
[32,  1400] loss: 0.204
[32,  1500] loss: 0.192
Mean Accuracy :  90.44
Mean Accuracy :  90.67
[33,   100] loss: 0.196
[33,   200] loss: 0.198
[33,   300] loss: 0.207
[33,   400] loss: 0.203
[33,   500] loss: 0.198
[33,   600] loss: 0.204
[33,   700] loss: 0.191
Mean Accuracy :  90.49
[33,   800] loss: 0.212
[33,   900] loss: 0.182
[33,  1000] loss: 0.212
[33,  1100] loss: 0.201
[33,  1200] loss: 0.190
[33,  1300] loss: 0.194

[46,  1100] loss: 0.171
[46,  1200] loss: 0.188
[46,  1300] loss: 0.197
[46,  1400] loss: 0.181
[46,  1500] loss: 0.185
Mean Accuracy :  90.29
Mean Accuracy :  90.89
[47,   100] loss: 0.189
[47,   200] loss: 0.186
[47,   300] loss: 0.183
[47,   400] loss: 0.186
[47,   500] loss: 0.171
[47,   600] loss: 0.172
[47,   700] loss: 0.192
Mean Accuracy :  90.76
[47,   800] loss: 0.196
[47,   900] loss: 0.185
[47,  1000] loss: 0.171
[47,  1100] loss: 0.157
[47,  1200] loss: 0.175
[47,  1300] loss: 0.174
[47,  1400] loss: 0.167
[47,  1500] loss: 0.183
Mean Accuracy :  90.44
Mean Accuracy :  90.45
[48,   100] loss: 0.170
[48,   200] loss: 0.173
[48,   300] loss: 0.196
[48,   400] loss: 0.182
[48,   500] loss: 0.192
[48,   600] loss: 0.188
[48,   700] loss: 0.173
Mean Accuracy :  91.24
[48,   800] loss: 0.170
[48,   900] loss: 0.155
[48,  1000] loss: 0.179
[48,  1100] loss: 0.176
[48,  1200] loss: 0.173
[48,  1300] loss: 0.177
[48,  1400] loss: 0.185
[48,  1500] loss: 0.173
Mean Accuracy :  90.76

## Test

In [8]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

Accuracy of plane : 92.000000 %
Accuracy of   car : 91.800000 %
Accuracy of  bird : 89.000000 %
Accuracy of   cat : 83.800000 %
Accuracy of  deer : 92.800000 %
Accuracy of   dog : 87.300000 %
Accuracy of  frog : 94.100000 %
Accuracy of horse : 91.800000 %
Accuracy of  ship : 95.000000 %
Accuracy of truck : 94.100000 %
Mean Accuracy :  91.17
